Code for inference on a trained YOLOv8n model

In [ ]:
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

In [ ]:
# Load YOLOv8n model from .pt file
model = YOLO(r"D:\runs\detect\train\weights\best.pt")

# Load image for inference
image_path = r"D:\test\images\filename.jpg"

image = cv2.imread(image_path)

In [ ]:
# Run inference
results = model(image)

# Visualize results
annotated_frame = results[0].plot()  # Draws boxes and labels on the image

# Show the image
cv2.imshow("Detection", annotated_frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Optionally, save the annotated image
cv2.imwrite("img_pred.jpg", annotated_frame)

In [ ]:
metrics = model.val(
    data=r"D:\data.yaml",  # change path in yaml file
    conf=0.25,                 # confidence threshold
    iou=0.5,                   # IoU threshold for matching
    split="test",              # 'valid' or 'test' split
    plots=True                # Save confusion matrix and other plots
)

# Access confusion matrix
conf_matrix = metrics.confusion_matrix
print("Confusion Matrix:\n", conf_matrix.matrix)


📊 YOLOv8 Evaluation Metrics Class

This class is designed for computing and storing evaluation metrics for YOLOv8 models.

    metrics.save_dir

    Attributes:
        save_dir (Path): A path to the directory where the output plots will be saved.
        plot (bool): A flag that indicates whether to plot precision-recall curves for each class.
        names (dict): A dictionary of class names.
        box (Metric): An instance of the Metric class for storing detection results.
        speed (dict): A dictionary for storing execution times of different parts of the detection process.
        task (str): The task type, set to 'detect'.



    metrics.box

🧬 Attributes

| Attribute        | Type | Description                                                        |
| ---------------- | ---- | ------------------------------------------------------------------ |
| `p`              | list | Precision for each class. Shape: `(nc,)`                           |
| `r`              | list | Recall for each class. Shape: `(nc,)`                              |
| `f1`             | list | F1 score for each class. Shape: `(nc,)`                            |
| `all_ap`         | list | AP scores for all classes across IoU thresholds. Shape: `(nc, 10)` |
| `ap_class_index` | list | Index of class for each AP score. Shape: `(nc,)`                   |
| `nc`             | int  | Number of classes                                                  |

⚙️ Methods

| Method            | Returns | Description                                                         |
| ----------------- | ------- | ------------------------------------------------------------------- |
| `ap50()`          | `list`  | AP at IoU = 0.5 for all classes. Shape: `(nc,)` or `[]`             |
| `ap()`            | `list`  | AP at IoUs from 0.5 to 0.95 for all classes. Shape: `(nc,)` or `[]` |
| `mp()`            | `float` | Mean precision across all classes                                   |
| `mr()`            | `float` | Mean recall across all classes                                      |
| `map50()`         | `float` | Mean AP at IoU = 0.5 across all classes                             |
| `map75()`         | `float` | Mean AP at IoU = 0.75 across all classes                            |
| `map()`           | `float` | Mean AP from IoU 0.5 to 0.95 across all classes                     |
| `mean_results()`  | `tuple` | Returns tuple: `(mp, mr, map50, map)`                               |
| `class_result(i)` | `tuple` | Returns class-specific metrics: `(p[i], r[i], ap50[i], ap[i])`      |
| `maps()`          | `list`  | Returns mAP scores for each class. Shape: `(nc,)`                   |
| `fitness()`       | `float` | Returns model fitness score based on weighted metrics               |
| `update(results)` | `None`  | Updates internal metrics with new evaluation results                |


Calculate Precision, Recall, mAP50, mAP50-95

In [ ]:
results_dict = metrics.results_dict
# print(results_dict)
results_keys = metrics.results_dict.keys()
print(results_keys, '\n')

precision = results_dict['metrics/precision(B)']*100
recall = results_dict['metrics/recall(B)']*100
mAP50 = results_dict['metrics/mAP50(B)']*100
mAP50_95 = results_dict['metrics/mAP50-95(B)']*100

print(f'Precision: {precision:.1f}')
print(f'Recall: {recall:.1f}')
print(f'mAP50: {mAP50:.1f}')
print(f'mAP50-95: {mAP50_95:.1f}')


Plot Confusion Matrix Manually

In [ ]:
%matplotlib inline

font_size = 12
fig_size = 6

cm = metrics.confusion_matrix

# Plot confusion matrix manually
fig, ax = plt.subplots(figsize=(fig_size, fig_size))
im = ax.imshow(cm.matrix, interpolation='nearest', cmap=plt.cm.Blues)
# plt.colorbar(im)

# change class according to dataset
classes = ['bolt', 'tie-plate', 'missing bolt', 'clip', 'missing clip', 'Defect FP', 'Non-defect FP', 'background']

ax.set(xticks=range(len(classes)),
       yticks=range(len(classes)),
       xticklabels=classes, 
       yticklabels=classes,
       )

ax.set_xlabel('True', fontsize=font_size, fontweight='bold')
ax.set_ylabel('Predicted', fontsize=font_size, fontweight='bold')
ax.set_title('Confusion Matrix', fontsize=font_size, fontweight='bold')

# Rotate tick labels
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor", fontsize=font_size)
plt.setp(ax.get_yticklabels(), fontsize=font_size)

# Annotate cells with values
fmt = 'd'
thresh = cm.matrix.max() / 2.
for i in range(cm.matrix.shape[0]):
    for j in range(cm.matrix.shape[1]):
        if int(cm.matrix[i, j]) > 0:
            ax.text(j, i, format(int(cm.matrix[i, j]), fmt),
                    ha="center", va="center",
                    color="white" if cm.matrix[i, j] > thresh else "black",
                    fontsize=font_size)  # <-- change this font size

fig.tight_layout()
plt.savefig("custom_confusion_matrix.png", dpi=500)
# plt.show()


Plot normalized confusion matrix

In [ ]:
# Normalize the confusion matrix row-wise
matrix = cm.matrix.astype('float')
normalized_matrix = matrix / matrix.sum(axis=0, keepdims=True)

# Plot
fig, ax = plt.subplots(figsize=(fig_size, fig_size))
im = ax.imshow(normalized_matrix, interpolation='nearest', cmap=plt.cm.Blues)

# Set ticks and labels
ax.set(
    xticks=range(len(classes)),
    yticks=range(len(classes)),
    xticklabels=classes,
    yticklabels=classes
)
ax.set_xlabel('True', fontsize=font_size, fontweight='bold')
ax.set_ylabel('Predicted', fontsize=font_size, fontweight='bold')
ax.set_title('Normalized Confusion Matrix', fontsize=font_size, fontweight='bold')

# Set tick label font sizes
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor", fontsize=font_size)
plt.setp(ax.get_yticklabels(), fontsize=font_size)

# Annotate each cell only if value > 0
fmt = '.2f'
thresh = normalized_matrix.max() / 2.
for i in range(normalized_matrix.shape[0]):
    for j in range(normalized_matrix.shape[1]):
        val = normalized_matrix[i, j]
        if val > 0.01:
            ax.text(j, i, format(val, fmt),
                    ha="center", va="center",
                    color="white" if val > thresh else "black",
                    fontsize=font_size)

fig.tight_layout()
plt.savefig("normalized_confusion_matrix.png", dpi=500)
# plt.show()
